In [1]:
%config IPCompleter.greedy=True
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys, re, math, datetime as dt, pandas as pd, numpy as np, time
import logging
import matplotlib.pyplot as plt
from pathlib import Path
from string import Template
from IPython.display import display, HTML

logging.basicConfig(format='%(asctime)s [%(name)s:%(lineno)d:%(funcName)s] [%(levelname)s] %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_colwidth', 5000)
pd.set_option('display.width', 5000)

def display_df(df):
    display(df.head(4))
    print(df.shape)

In [43]:
import fastf1 as f1

session = f1.get_session(2023, 4, 'R')
cache_dir = f"/Users/hannahwang/Projects/formula_1/src/fastf1_cache"
f1.Cache.enable_cache(cache_dir=cache_dir)
# schedule = f1.get_event_schedule(2023)
# # print(session.event)
# display_df(schedule)
# schedule.dtypes
session

2023 Season Round 4: Azerbaijan Grand Prix - Race

In [44]:
session.load()

core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v3.1.6]
2024-01-06 15:40:43,301 [fastf1.fastf1.core:1248:load] [INFO] Loading data for Azerbaijan Grand Prix - Race [v3.1.6]
req            INFO 	Using cached data for session_info
2024-01-06 15:40:43,305 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_info
req            INFO 	Using cached data for driver_info
2024-01-06 15:40:43,308 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
2024-01-06 15:40:45,171 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
2024-01-06 15:40:45,174 [fastf1.fastf1.req:408:_cached_api_request] [INFO] Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
2024-01-06 15:40:45,176 [fastf1.fastf1.req:408:_cached_api_request] [INFO] U

In [45]:
fastest = session.laps.pick_driver("VER").pick_fastest()
fastest

Time                      0 days 02:35:46.586000
Driver                                       VER
DriverNumber                                   1
LapTime                   0 days 00:01:44.232000
LapNumber                                   51.0
Stint                                        2.0
PitOutTime                                   NaT
PitInTime                                    NaT
Sector1Time               0 days 00:00:36.807000
Sector2Time               0 days 00:00:42.294000
Sector3Time               0 days 00:00:25.131000
Sector1SessionTime        0 days 02:34:39.165000
Sector2SessionTime        0 days 02:35:21.459000
Sector3SessionTime        0 days 02:35:46.590000
SpeedI1                                    219.0
SpeedI2                                    229.0
SpeedFL                                    329.0
SpeedST                                    325.0
IsPersonalBest                              True
Compound                                    HARD
TyreLife            

In [179]:
tel_data = fastest.get_telemetry()
print(tel_data.dtypes)
tel_data.head()

Date                      datetime64[ns]
SessionTime              timedelta64[ns]
DriverAhead                       object
DistanceToDriverAhead            float64
Time                     timedelta64[ns]
RPM                                int64
Speed                              int64
nGear                              int64
Throttle                           int64
Brake                               bool
DRS                                int64
Source                            object
Distance                         float64
RelativeDistance                 float64
Status                            object
X                                  int64
Y                                  int64
Z                                  int64
dtype: object


,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
2,2023-04-30 12:35:03.824,0 days 02:34:02.354000,,216.202222,0 days 00:00:00,11589,328,8,100,False,0,interpolation,0.000473,7.919993e-08,OnTrack,1037,-602,-245
3,2023-04-30 12:35:03.851,0 days 02:34:02.381000,,216.202222,0 days 00:00:00.027000,11610,328,8,100,False,0,pos,2.460428,4.118722e-04,OnTrack,1061,-593,-245
4,2023-04-30 12:35:03.945,0 days 02:34:02.475000,,216.202222,0 days 00:00:00.121000,11653,328,8,100,False,0,car,11.024444,1.845476e-03,OnTrack,1140,-559,-244
5,2023-04-30 12:35:04.031,0 days 02:34:02.561000,,216.202222,0 days 00:00:00.207000,11578,328,8,100,False,0,pos,18.859328,3.157025e-03,OnTrack,1212,-530,-245
6,2023-04-30 12:35:04.146,0 days 02:34:02.676000,11,216.202222,0 days 00:00:00.322000,11504,328,8,100,False,0,car,29.337778,4.911103e-03,OnTrack,1308,-489,-245


In [180]:
tel_data['Specific_Time'] = tel_data.Time.map(lambda x: f"{int(x.seconds/3600)}:{int(x.seconds)}:{x.seconds%3600}.{str(x.microseconds).zfill(6)}")

In [181]:
tel_data.head(1)

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,Specific_Time
2,2023-04-30 12:35:03.824,0 days 02:34:02.354000,,216.202222,0 days,11589,328,8,100,False,0,interpolation,0.000473,7.919993e-08,OnTrack,1037,-602,-245,0:0:0.000000


In [182]:
def rotate_point(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    # angle = angle/360*2*np.pi
    if point[0] is None or point[1] is None:
        return

    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def rangeof(points):
    try:
        max_x = max(points[0])
        min_x = min(points[0])
        max_y = max(points[1])
        min_y = min(points[1])
    except TypeError:
        print("Nonetype")
        return

    return max_x-min_x, max_y-min_y
def rotate_90(points, center_x, center_y):
    for index in range(len(points[0])):
        if points[0][index] is None:
            continue
        points[0][index], points[1][index] = rotate_point([center_x, center_y],
                                                    [points[0][index], points[1][index]],
                                                    np.pi/2)

    return points

def flip_v(points, center_x, center_y):
    for index in range(len(points[0])):
        if points[0][index] is None:
            continue
        x = points[0][index]
        points[0][index] = -1*(x-center_x) + center_x

        y = points[1][index]
        points[1][index] = -1*(y-center_y) + center_y

    return points

import numpy as np
def rotate(xy, *, angle):
    rot_mat = np.array([[np.cos(angle), np.sin(angle)],
                        [-np.sin(angle), np.cos(angle)]])
    return np.matmul(xy, rot_mat)

In [183]:
corners = session.get_circuit_info().corners
corners = [corners.X, corners.Y]

corners

[0      3689.037400
 1      2384.066914
 2     -5565.825568
 3     -4862.665177
 4     -7728.697050
 5     -7617.318818
 6    -10710.063799
 7    -11409.887894
 8    -11606.255350
 9    -11886.309274
 10   -12352.278964
 11   -12573.090272
 12   -16186.999703
 13   -16888.824375
 14   -16846.430312
 15   -13660.610624
 16   -12714.251049
 17   -11029.533801
 18   -10524.590291
 19    -8077.310039
 Name: X, dtype: float64,
 0       631.318973
 1      3686.071452
 2       374.242552
 3     -1723.162008
 4     -3377.074091
 5     -3926.026038
 6     -6235.337257
 7     -4438.977048
 8     -4286.033174
 9     -4199.071400
 10    -4129.880227
 11    -3645.190016
 12    -5197.330621
 13    -7335.751780
 14    -9555.895632
 15   -11108.575389
 16    -9481.441337
 17    -7796.238534
 18    -6346.594211
 19    -4403.674075
 Name: Y, dtype: float64]

In [184]:
session.get_circuit_info().corners

,X,Y,Number,Letter,Angle,Distance
0,3689.037400,631.318973,1,,-24.903645,295.860282
1,2384.066914,3686.071452,2,,77.316771,633.238168
2,-5565.825568,374.242552,3,,145.179332,1499.693679
3,-4862.665177,-1723.162008,4,,-26.596082,1719.960465
4,-7728.697050,-3377.074091,5,,-199.974573,2064.211037
5,-7617.318818,-3926.026038,6,,-202.275097,2122.106222
6,-10710.063799,-6235.337257,7,,68.563570,2503.927252
7,-11409.887894,-4438.977048,8,,25.432318,2701.913133
8,-11606.255350,-4286.033174,9,,66.839444,2724.742425
9,-11886.309274,-4199.071400,10,,81.385303,2758.002530


In [185]:
tel_data.head(1)

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,Specific_Time
2,2023-04-30 12:35:03.824,0 days 02:34:02.354000,,216.202222,0 days,11589,328,8,100,False,0,interpolation,0.000473,7.919993e-08,OnTrack,1037,-602,-245,0:0:0.000000


In [186]:
max_speed = max(tel_data.Speed.tolist())

x_co = []
y_co = []

x = 16
y = 17
speed = 7

for row in tel_data.itertuples():
    if max_speed == row[speed]:
        x_co.append(row[x])
        y_co.append(row[y])

max_speed_points = [x_co, y_co]


measure = "Brake" # Speed, Brake, Throttle, nGear, RPM
unit = ""

points = [tel_data['X'].tolist(), tel_data['Y'].tolist()]
metric = tel_data[measure]
title = "Fastest Lap"
subtitle = f"{measure} {unit}"
colorscale = 'hot'


import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

screen_width = 1750


dim_x, dim_y = rangeof(points)

max_x = max(points[0])
min_x = min(points[0])
max_y = max(points[1])
min_y = min(points[1])
center_x = (max_x + min_x) / 2
center_y = (max_y + min_y) / 2
# corners = flip_v(corners, center_x, center_y)

if dim_x < dim_y:
    points = rotate_90(points, center_x, center_y)
    brakepoints = rotate_90(brakepoints, center_x, center_y)
    max_speed_points = rotate_90(max_speed_points, center_x, center_y)
    corners = rotate_90(corners, center_x, center_y)

width = screen_width*2/3

dim_x, dim_y = rangeof(points)
dim_y = dim_y/dim_x * width
dim_x = width

size_metric = int(width*.0075)

tel_data['Lap_Time'] = tel_data.Time.map(lambda x: f"{int(x.seconds / 3600)}:{int(x.seconds)}:{x.seconds % 3600}.{str(x.microseconds).zfill(6)}")

# create figure
fig = px.scatter(tel_data, x=points[0], y=points[1],
                 color_continuous_scale=colorscale,
                 color=measure,
                 size=[1]*len(points[0]),
                 size_max=size_metric,
                 opacity=0.8,
                 # title=f"{title} by {subtitle}",
                 # height=dim_y, width=dim_x,
                 hover_data=['RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Status', 'Lap_Time'],

                 )

fig_brakes = go.Scatter(x=brakepoints[0], y=brakepoints[1],
                        mode='markers',
                        name='brake points',
                        opacity=0.25,
                        marker=dict(color="#FFA1F2",
                                    size=size_metric*2.25),
                        hoverinfo='skip'
                        )
fig_max_speed = go.Scatter(x=max_speed_points[0], y=max_speed_points[1],
                           mode='markers',
                           name='max speed',
                           opacity=.9,
                           marker=dict(color="#FF006C",
                                       size=size_metric*1.5,
                                       ),
                           hoverinfo='skip'
                           )

fig_corners = go.Scatter(x=corners[0], y=corners[1],
                           mode='markers+text',
                           name='corners',
                           opacity=1,
                           marker=dict(color="rgb(20,20,20)",
                                       size=size_metric*2,
                                       ),
                           text=session.get_circuit_info().corners.Number,
                           hoverinfo='skip',
                           textposition='middle center',
                           textfont=dict(color='white', size=8),
                           # line=dict(color='#52BCA3', width=1, dash='dash'),

                           )

fig.add_trace(fig_brakes)
fig.add_trace(fig_max_speed)
fig.add_trace(fig_corners)

fig.update_layout(xaxis_visible=False, yaxis_visible=False,
                  legend=dict(
                      orientation="h",
                      yanchor="bottom",
                      y=1.02,
                      xanchor="right",
                      x=1,
                      font=dict(
                          color="rgb(230,230,230)"
                      ),
                              ),
                  plot_bgcolor='#252525',
                  autosize=False,
                  height=dim_y, width=dim_x,
                  margin=dict(
                      l=15,
                      r=15,
                      b=15,
                      t=15,
                      pad=4
                  ),
                  paper_bgcolor="rgb(10,10,10)",
                  )

fig.update_coloraxes(
    colorbar=dict(
        title=dict(
            text="Speed [km/h]",
            font_color='white',
        ),
        y=0.5,
        showticklabels=True,
        ticks='outside',
        tickfont_color='white',
        tickcolor='white',
        ticklabelstep=2,
        dtick=10
    ),
)

fig.update_traces({'marker.opacity':1, 'marker.line.width':0})

fig.show()

In [24]:
measure = "Speed"
unit = "[km/h]"

points = [tel_data['X'].tolist(), tel_data['Y'].tolist()]
metric = tel_data[measure]
title = "Fastest Lap"
subtitle = f"{measure} {unit}"
colorscale = 'hot'

In [172]:
tel_data = session.laps.pick_driver("HAM").copy()


In [173]:
tel_data.head()
tel_data.dtypes

Time                  timedelta64[ns]
Driver                         object
DriverNumber                   object
LapTime               timedelta64[ns]
LapNumber                     float64
Stint                         float64
PitOutTime            timedelta64[ns]
PitInTime             timedelta64[ns]
Sector1Time           timedelta64[ns]
Sector2Time           timedelta64[ns]
Sector3Time           timedelta64[ns]
Sector1SessionTime    timedelta64[ns]
Sector2SessionTime    timedelta64[ns]
Sector3SessionTime    timedelta64[ns]
SpeedI1                       float64
SpeedI2                       float64
SpeedFL                       float64
SpeedST                       float64
IsPersonalBest                 object
Compound                       object
TyreLife                      float64
FreshTyre                        bool
Team                           object
LapStartTime          timedelta64[ns]
LapStartDate           datetime64[ns]
TrackStatus                    object
Position    

In [174]:
import datetime
tel_data['Seconds'] = tel_data.LapTime.map(lambda x: x/datetime.timedelta(seconds=1))


In [175]:
tel_data.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Seconds
708,0 days 01:04:55.432000,HAM,44,0 days 00:01:53.406000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:43.889000,0 days 00:00:25.759000,NaT,0 days 01:04:29.757000,0 days 01:04:55.522000,194.0,212.0,321.0,315.0,False,MEDIUM,1.0,True,Mercedes,0 days 01:03:01.758000,2023-04-30 11:04:03.228,2,5.0,False,,False,False,113.406
709,0 days 01:06:43.597000,HAM,44,0 days 00:01:48.165000,2.0,1.0,NaT,NaT,0 days 00:00:38.632000,0 days 00:00:43.752000,0 days 00:00:25.781000,0 days 01:05:34.089000,0 days 01:06:17.841000,0 days 01:06:43.622000,181.0,205.0,320.0,317.0,True,MEDIUM,2.0,True,Mercedes,0 days 01:04:55.432000,2023-04-30 11:05:56.902,1,5.0,False,,False,True,108.165
710,0 days 01:08:31.199000,HAM,44,0 days 00:01:47.602000,3.0,1.0,NaT,NaT,0 days 00:00:38.197000,0 days 00:00:43.760000,0 days 00:00:25.645000,0 days 01:07:21.819000,0 days 01:08:05.579000,0 days 01:08:31.224000,191.0,206.0,338.0,319.0,True,MEDIUM,3.0,True,Mercedes,0 days 01:06:43.597000,2023-04-30 11:07:45.067,1,5.0,False,,False,True,107.602
711,0 days 01:10:18.959000,HAM,44,0 days 00:01:47.760000,4.0,1.0,NaT,NaT,0 days 00:00:38.228000,0 days 00:00:43.985000,0 days 00:00:25.547000,0 days 01:09:09.452000,0 days 01:09:53.437000,0 days 01:10:18.984000,196.0,206.0,339.0,318.0,False,MEDIUM,4.0,True,Mercedes,0 days 01:08:31.199000,2023-04-30 11:09:32.669,1,5.0,False,,False,True,107.760
712,0 days 01:12:06.750000,HAM,44,0 days 00:01:47.791000,5.0,1.0,NaT,NaT,0 days 00:00:38.273000,0 days 00:00:44.095000,0 days 00:00:25.423000,0 days 01:10:57.257000,0 days 01:11:41.352000,0 days 01:12:06.775000,194.0,205.0,339.0,319.0,False,MEDIUM,5.0,True,Mercedes,0 days 01:10:18.959000,2023-04-30 11:11:20.429,1,5.0,False,,False,True,107.791


In [176]:
tel_data = tel_data.sort_values(by='Seconds', ascending=True)
tel_data.LapNumber.iloc[0]

48.0